# 8. 감성 분석에 머신러닝 적용

# 8.1 텍스트 처리용 IMDb 영화 리뷰 데이터 준비

### 8.1.1 영화 리뷰 데이터셋 구하기

IMDB 영화 리뷰 데이터셋은 [http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz](http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz)에서 내려받을 수 있다. 다운로드된 후 파일 압축을 해제한다.

A) 리눅스(Linux)나 macOS를 사용하면 새로운 터미널(Terminal) 윈도우를 열고 `cd` 명령으로 다운로드 디렉터리로 이동하여 다음 명령을 실행. 

`tar -zxf aclImdb_v1.tar.gz`

B) 윈도(Windows)를 사용하면 7-Zip(http://www.7-zip.org) 같은 무료 압축 유틸리티를 설치하여 다운로드한 파일의 압축을 풀 수 있다.

In [1]:
import os
import sys
import tarfile
import time
import urllib.request


source = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
target = 'aclImdb_v1.tar.gz'


def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = progress_size / (1024.**2 * duration)
    percent = count * block_size * 100. / total_size

    sys.stdout.write("\r%d%% | %d MB | %.2f MB/s | %d sec elapsed" %
                    (percent, progress_size / (1024.**2), speed, duration))
    sys.stdout.flush()


if not os.path.isdir('aclImdb') and not os.path.isfile('aclImdb_v1.tar.gz'):
    urllib.request.urlretrieve(source, target, reporthook)

In [2]:
# Gzip 압축된 타볼 파일의 압축을 풀 수 있다.
if not os.path.isdir('aclImdb'):

    with tarfile.open(target, 'r:gz') as tar:
        tar.extractall()

### 8.1.2 영화 리뷰 데이터셋을 더 간편한 형태로 전처리

데이터셋의 압축을 푼 후 압축이 풀린 개개의 텍스트 문서를 하나의 CSV 파일로 합친다. 영화 리뷰를 읽어 하나의 판다스 DataFrame 객체로 만든다.

In [3]:
!pip install pyprind

In [4]:
import pyprind
import pandas as pd
import os

# `basepath`를 압축 해제된 영화 리뷰 데이터셋이 있는 디렉토리로 바꾼다.
basepath = 'aclImdb'

labels = {'pos': 1, 'neg': 0} # 정수 클래스 레이블 (1=긍정, 0=부정)
pbar = pyprind.ProgBar(50000) # 읽어들일 문서 개수
df = pd.DataFrame()
for s in ('test', 'train'): # aclImdb 디렉터리의 하위 디렉터리 train, test
    for l in ('pos', 'neg'): # 그 아래 pos와 neg 디렉터리에서
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file),  # 개별 텍스트 파일을 읽는다
                      'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], 
                           ignore_index=True)
            pbar.update()
df.columns = ['review', 'sentiment']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:56


합친 데이터셋의 클래스 레이블이 순서대로 나열되어 있다. np.random 모듈의 permutation 함수를 사용하여 이 데이터프레임을 섞는다.

In [5]:
import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('movie_data.csv', index=False, encoding='utf-8')

df.head(3)

,review,sentiment
11841,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
19602,OK... so... I really like Kris Kristofferson a...,0
45519,"***SPOILER*** Do not read this, if you think a...",0


In [6]:
df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [7]:
df.shape

(50000, 2)

# 8.2 BoW 모델 소개
1. 전체 문서에 대해 고유한 토큰, 예를 들어 단어로 이루어진 어휘 사전을 만든다.
2. 특정 문서에 각 단어가 얼마나 자주 등장하는지 헤어려 문서의 특성 벡터를 만든다
* 각 문서에 있는 고유한 단어는 BoW 어휘 사전에 있는 모든 단어의 일부분에 지나지 않으므로 특성 벡터는 대부분 0으로 채워진다.
    * **희소 행렬**

### 8.2.1 단어를 특정 벡터로 변환
* 사이킷런 CountVectorizer 클래스 - 각각의 문서에 있는 단어 카운트 기반으로 BoW 모델 구현


In [8]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)

In [9]:
# 어휘 사전 내용 - 고유 단어와 정수 인덱스
count.vocabulary_

{'and': 0,
 'is': 1,
 'one': 2,
 'shining': 3,
 'sun': 4,
 'sweet': 5,
 'the': 6,
 'two': 7,
 'weather': 8}

특성 벡터의 각 인덱스는 CountVectorizer의 어휘 사전 딕셔너리에 저장된 정수 값에 해당된다. 예를 들어 인덱스 0에 있는 첫 번째 특성은 ‘and’ 단어의 카운트를 의미한다. 인덱스 1에 있는 (특성 벡터의 두 번째 열) 단어 ‘is’는 세 문장에 모두 등장한다. 특성 벡터의 이런 값들을 단어 빈도(term frequency) 라고도 부른다. 문서 d에 등장한 단어 t의 횟수를 *tf (t,d)*와 같이 쓴다.

In [10]:
print(bag.toarray())

"""
[[0 1 0 1 1 0 1 0 0]   : and=0, is=1, one=0, shining=1, sun=1, sweet=0, the=1, two=0, weather=0
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]
 """

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


'\n[[0 1 0 1 1 0 1 0 0]   : and=0, is=1, one=0, shining=1, sun=1, sweet=0, the=1, two=0, weather=0\n [0 1 0 0 0 1 1 0 1]\n [2 3 2 1 1 1 2 1 1]]\n '

### 8.2.2 tf-idf를 사용하여 단어 적합성 평가
* 특정 벡터에서 자주 등장하는 단어의 가중치를 낮추는 기법
* tf-idf는 단어 빈도와 역문서 빈도(inverse document frequency)의 곱으로 정의

$$\text{tf-idf}(t,d)=\text{tf (t,d)}\times \text{idf}(t,d)$$

* tf(t, d)는 단어 빈도, idf(t, d)는 역분서 빈도.

$$\text{idf}(t,d) = \text{log}\frac{n_d}{1+\text{df}(d, t)},$$

$n_d$는 전체 문서 개수이고, df(d, t)는 단어 t가 포함된 문서의 d의 개수이다. 분모에 상수 1을 추가하는 것은 선택 사항으로, 훈련 샘플에 한 번도 등장하지 않는 단어가 있는 경우 분모가 0이 되지 않게 만든다. log는 문서 빈도 df(d, t)가 낮을 때 역문서 빈도 값이 너무 커지지 않도록 만든다.

* 사이킷런 CountVectorizer 클래스에서 만든 단어 빈도를 입력받아 TfidfTransformer 클래스를 사용하여 tf-idf로 변환.



In [11]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


CountVectorizer에서 확인한 단어 빈도에서는 'is'가 가장 많이 나타났었다. 동일한 특성 벡터를 tf-idf로 변환하면 단어 'is'는 비교적 작은 tf-idf를 가진다(0.45). 이 단어는 첫 번째와 두 번째 문서에도 나타나므로 판별에 유용한 정보를 가지고 있지 않을 것이다.

수동으로 특성 벡터에 있는 각 단어의 tf-idf를 계산해보면 `TfidfTransformer`가 위에서 정의한 표준 tf-idf 공식과 조금 다르게 계산된다. 
* 사이킷런에 구현된 역문서 빈도 공식

$$\text{idf} (t,d) = log\frac{1 + n_d}{1 + \text{df}(d, t)}$$

* 사이킷런 tf-idf 공식

$$\text{tf-idf}(t,d) = \text{tf}(t,d) \times (\text{idf}(t,d)+1)$$

'+1'을 더한 것은 모든 문서에 등장하는 단어가 가중치 0이 되는 것(즉, idf(t, d)=log(1)=0)을 막기 위해서이다.

일반적으로 tf-idf를 계산하기 전에 단어 빈도(tf)를 정규화하지만 `TfidfTransformer` 클래스는 tf-idf를 직접 정규화한다. 사이킷런의 TfidfTransformer는 기본적으로 L2 정규화를 적용(norm='l2'). 정규화되지 않은 특성 벡터 v를 L2로 나누면 길이가 1인 벡터가 반환된다.

$$v_{\text{norm}} = \frac{v}{||v||_2} = \frac{v}{\sqrt{v_{1}^{2} + v_{2}^{2} + \dots + v_{n}^{2}}} = \frac{v}{\big (\sum_{i=1}^{n} v_{i}^{2}\big)^\frac{1}{2}}$$

$\text{idf}("is", d3) = log \frac{1+3}{1+3} = 0$

$\text{tf-idf}("is",d3)= 3 \times (0+1) = 3$

In [12]:
tf_is = 3
n_docs = 3
idf_is = np.log((n_docs+1) / (3+1))
tfidf_is = tf_is * (idf_is + 1)
print('idf "is" = %d' % idf_is)
print('tf-idf of term "is" = %.2f' % tfidf_is)

idf "is" = 0
tf-idf of term "is" = 3.00


세 번째 문서에 있는 모든 단어에 대해 이런 계산을 반복하면 tf-idf 벡터 [3.39, 3.0, 3.39, 1.29, 1.29, 1.29, 2.0, 1.69, 1.29]를 얻는다. 이 특성 벡터의 값은 앞서 사용했던 TfidfTransformer에서 얻은 값과 다르다. tf-idf 계산에서 빠트린 마지막 단계는 다음과 같은 L2-정규화이다.

$$\text{tfi-df}_{norm} = \frac{[3.39, 3.0, 3.39, 1.29, 1.29, 1.29, 2.0 , 1.69, 1.29]}{\sqrt{[3.39^2, 3.0^2, 3.39^2, 1.29^2, 1.29^2, 1.29^2, 2.0^2 , 1.69^2, 1.29^2]}}$$

$$=[0.5, 0.45, 0.5, 0.19, 0.19, 0.19, 0.3, 0.25, 0.19]$$

$$\Rightarrow \text{tfi-df}_{norm}("is", d3) = 0.45$$

In [13]:
tfidf = TfidfTransformer(use_idf=True, norm=None, smooth_idf=True)
raw_tfidf = tfidf.fit_transform(count.fit_transform(docs)).toarray()[-1]
l2_tfidf = raw_tfidf / np.sqrt(np.sum(raw_tfidf**2))
print(raw_tfidf)
print('= {0}'.format(l2_tfidf))

[3.39 3.   3.39 1.29 1.29 1.29 2.   1.69 1.29]
= [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]


### 8.2.3 텍스트 데이터 정제

In [14]:
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

HTML 마크업은 구두점과 글자가 아닌 문자가 포함되어 있다. HTML 마크업에는 유용한 의미가 많지 않지만 구두점은 특정 NLP 문제에서 쓸모있는 추가 정보가 될 수 있다. 

간단하게 하기위해 :) 와 같은 이모티콘 문자를 제외하고 모든 구두점 기호를 삭제한다. 이런 이모티콘은 확실히 감성 분석에 유용하다. 

In [15]:
import re

def preprocessor(text):
    text = re.sub('<[^>]*>', '', text) # 모든 HTML 마크업 삭제
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text) # 마크업을 제거한 후 이모티콘을 찾아 저장
    text = (re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')) # 텍스트를 소문자로 바꾸고 [\W]+를 사용하여 텍스트에서 단어가 아닌 문자를 모두 제거
    return text

In [16]:
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

In [17]:
preprocessor("</a>This :) is :( a test :-)!")

'this is a test :) :( :)'

In [18]:
df['review'] = df['review'].apply(preprocessor)

### 3.2.4 문서를 토큰으로 나누기

영화 리뷰 데이터셋을 전처리한 후에는 어떻게 텍스트 문서를 낱개의 토큰으로 나눌지 생각해야 한다. 문서를 토큰화하는 한 가지 방법은 공백 문자를 기준으로 개별 단어로 나누는 것이다.

In [19]:
def tokenizer(text):
    return text.split()

tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

토큰화 방법 중에는 단어는 변하지 않는 기본 형태인 어간으로 바꾸는 **어간 추출**(stemming)이란 방법이 있다. 여러 가지 형태를 갖는 단어를 같은 어간으로 매핑할 수 있다. 초기 어간 추출 알고리즘은 포터 어간 추출기 알고리즘이라고 한다.

파이썬의 **NLTK** 패키지 - 어간 추출 알고리즘

In [20]:
# 포터 어간 추출 알고리즘
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

nltk 패키지의 PorterStemmer 클래스를 사용하여 단어의 어간으로 바꾸기 위해 tokenizer 함수를 변경했다. 이전 'running'이 어간 'run'으로 바뀌었다.

* 불용어(stop-word) 제거
    * 불용어 : 모든 종류의 텍스트에 아주 흔하게 등장하는 단어로, 'is', 'and', 'has', 'like' 등이 있다. 
    * tf-idf보다 기본 단어 빈도나 정규화된 단어 빈도를 사용할 때 유용하다.

In [21]:
import nltk
# 불용어 제거를 위해 NLTK 라이브러리에서 제공하는 179개의 불용어를 사용한다.
# 이 불용어는 nltk.download 함수를 호출하여 내려받는다.
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [22]:
# 불용어 집합을 내려받은 후 영어의 불용어를 불러들여 적용
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:] if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

# 8.3 문서 분류를 위한 로지스틱 회귀 모델 훈련

BoW 모델을 기반으로 영화 리뷰를 긍정과 부정 리뷰로 분류하는 로지스틱 회귀 모델을 훈련시킨다. 먼저 정제된 텍스트 문서가 저장된 DataFrame을 25,000개는 훈련 데이터셋으로 나누고 25,000개는 테스트 데이터셋으로 나눈다.

In [23]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [24]:
# GridSearchCV 객체에서 5-겹 계층별 교차 검증을 사용하여 로지스틱 회귀 모델에 대한 최적 매개변수 조합을 찾는다
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

# TfidfVectorizer : CountVectorizer와 TfidfTransformer를 하나로 합침
tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None) 

# 첫번째는 TfidfVectorizer의 기본 매개변수 셋팅(use_idf=True, norm='l2')을 사용하여 tf-idf 께산
# 두번째는 단어 빈도를 사용하여 모델을 훈련 시키기 위해 use_idf=False, norm=None 으로 설정
# 로지스틱 회귀 분류기는 penaly 매개변수를 통해 L1과 L2 규제를 적용하고,
# 규제 매개변수 C에 여러 값을 지정해 규제 강도를 비교
param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0, solver='liblinear'))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy', cv=5, n_jobs=-1)

* n_jobs 매개변수

컴퓨터에 있는 모든 CPU 코어를 사용해 그리드 서치의 속도를 높이려면 (`n_jobs=1` 대신) `n_jobs=-1`로 지정하는 것이 좋다. 일부 시스템에서는 멀티프로세싱을 위해 `n_jobs=-1`로 지정할 때 `tokenizer` 와 `tokenizer_porter` 함수의 직렬화에 문제가 발생할 수 있다. 이런 경우 `[tokenizer, tokenizer_porter]`를 `[str.split]`로 바꾸어 문제를 해결할 수 있다. 다만 `str.split`로 바꾸면 어간 추출을 하지 못한다.

**코랩을 사용할 경우에도 CPU 코어가 많지 않기 때문에 실행 시간이 오래 걸릴 수 있습니다.**

너무 오래 기다리기 어렵다면 데이터셋의 훈련 샘플의 수를 다음처럼 줄일 수 있습니다:

    X_train = df.loc[:2500, 'review'].values
    y_train = df.loc[:2500, 'sentiment'].values
    
훈련 세트 크기를 줄이는 것은 모델 성능을 감소시킵니다. 그리드에 지정한 매개변수를 삭제하면 훈련한 모델 수를 줄일 수 있습니다. 예를 들면 다음과 같습니다:

    param_grid = [{'vect__ngram_range': [(1, 1)],
                   'vect__stop_words': [stop, None],
                   'vect__tokenizer': [tokenizer],
                   'clf__penalty': ['l1', 'l2'],
                   'clf__C': [1.0, 10.0]},
                  ]

In [25]:
gs_lr_tfidf.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(random_state=0,
                                                           solver='liblinear'))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [1.0, 10.0, 100.0],
                          'clf__penalty': ['l1', 'l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've...
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd", 'your',
 

In [26]:
print('최적의 매개변수 조합: %s ' % gs_lr_tfidf.best_params_)

# 그리드 서치로 찾은 최상의 모델을 사용하여 훈련 데이터셋에 대한 모델의 5-겹 교차 검증 정확도와
# 테스트 데이터셋에 대한 분류 정확도를 출력
print('CV 정확도: %.3f' % gs_lr_tfidf.best_score_)
clf = gs_lr_tfidf.best_estimator_
print('테스트 정확도: %.3f' % clf.score(X_test, y_test))

최적의 매개변수 조합: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x7f208f3c0170>} 
CV 정확도: 0.897
테스트 정확도: 0.899


#### k-폴드 교차 검증과 GridSearchCV 객체

k-폴드 교차 검증으로 GridSearchCV 객체를 훈련하면 `best_score_` 속성은 최상의 모델에 대한 k-폴드 점수의 평균을 반환한다.

In [27]:
from sklearn.linear_model import LogisticRegression
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

np.random.seed(0)
np.set_printoptions(precision=6)
y = [np.random.randint(3) for i in range(25)]
X = (y + np.random.randn(25)).reshape(-1, 1)

cv5_idx = list(StratifiedKFold(n_splits=5, shuffle=False).split(X, y))
    
lr = LogisticRegression(random_state=123, multi_class='ovr', solver='lbfgs')
cross_val_score(lr, X, y, cv=cv5_idx)

array([0.4, 0.2, 0.6, 0.2, 0.4])

클래스 레이블에 해당하는 랜덤한 정수 데이터셋이 만들어진다. 그 다음 5-폴드의 인덱스(`cv5_idx`)를 cross_val_score 함수에 전달하여 5개의 정확도 점수를 받는다. 이 점수가 다섯 개의 테스트 폴드에 대한 정확도 값이다.

그 다음 `GridSearchCV` 객체를 사용해 동일한 5-폴드 인덱스(`cv5_idx`)를 전달.

In [28]:
from sklearn.model_selection import GridSearchCV

lr = LogisticRegression(solver='lbfgs', multi_class='ovr', random_state=1)
gs = GridSearchCV(lr, {}, cv=cv5_idx, verbose=3).fit(X, y) 

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END ..................................., score=0.400 total time=   0.0s
[CV 2/5] END ..................................., score=0.200 total time=   0.0s
[CV 3/5] END ..................................., score=0.600 total time=   0.0s
[CV 4/5] END ..................................., score=0.200 total time=   0.0s
[CV 5/5] END ..................................., score=0.400 total time=   0.0s


5-폴드에 대한 점수는 `cross_val_score`에서 얻은 것과 일치.

이제 fit 메서드를 호출한 후의 GridSearchCV 객체의 best_score_ 속성은 최상의 모델의 평균 정확도를 반환한다.

In [29]:
print(gs.best_score_)

lr = LogisticRegression(solver='lbfgs', multi_class='ovr', random_state=1)
print(cross_val_score(lr, X, y, cv=cv5_idx).mean())

0.36000000000000004
0.36000000000000004
